In [ ]:
# This colab notebook is loosely based off the following code/article:
# Yixing, F. (2020) Image classification via fine-tuning with EfficientNet. https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/
# Keshav, V., (2019) ImageNet from tensorflow_datasets. https://gist.github.com/vishal-keshav/0eac98fd44e2239ff5721196a9011829

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import tensorflow_datasets as tfds
import keras

In [ ]:
#Note that to load the dataset keras requires it to be downloaded manually and then uploaded to uploaded to be used here.
#Currently we have uploaded it to our drive. 
#We are using the tensor flow dataset from this link: https://www.tensorflow.org/datasets/catalog/imagenet2012_subset
#We downloaded the ILSVRC2012 dataset from here: http://www.image-net.org/download-images
dataset_name = "imagenet2012_subset"
imagenet_dataset_builder = tfds.builder(dataset_name)

manual_dataset_dir = "/content/gdrive/MyDrive/ImageNet Dataset/"
imagenet_download_config = tfds.download.DownloadConfig(
                                                manual_dir = manual_dataset_dir)
imagenet_dataset_builder.download_and_prepare(
                                    download_dir = manual_dataset_dir)
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name, split=["train", "validation"], with_info=True, as_supervised=True
)
NUM_CLASSES = ds_info.features["label"].num_classes
print(NUM_CLASSES)


In [ ]:
import tensorflow as tf
batch_size=500
size=(224,224)

ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

model = keras.applications.MobileNetV2()
model.compile(optimizer = 'RMSProp', loss="categorical_crossentropy", metrics=["accuracy"])

averages = []
for img,lbl in ds_test:
    temp_img = keras.applications.mobilenet_v2.preprocess_input(img)
    temp_l,temp_avg = model.evaluate(temp_img, lbl)
    averages.append(temp_avg)

16/16 [==============================] - 18s 1s/step - loss: 1.2614 - accuracy: 0.6940


In [ ]:
sum = 0
counter = 0
for i in averages:
  sum+=i
  counter+=1
avg = sum/counter
print("Average accuracy of MobileNetV2 on the ImageNet 2012 validation set:", avg)

Average accuracy of MobileNetV2 on the ImageNet 2012 validation set: 0.7063199985027313
